# Import Libraries and Construct Data

In [1]:
from __future__ import division
import pickle
import random
import numpy as np
from openrec.tf1.legacy import ImplicitModelTrainer
from openrec.tf1.legacy.utils import ImplicitDataset
from openrec.tf1.legacy.utils.evaluators import ImplicitEvalManager
from openrec.tf1.legacy.recommenders import CML, BPR
from openrec.tf1.legacy.utils.evaluators import AUC
from openrec.tf1.legacy.utils.samplers import PairwiseSampler

In [2]:
raw_data = dict()
raw_data['train_data'] = np.load("training_arr.npy")
raw_data['test_data_pos'] = np.load("unbiased-test_arr_pos.npy")
raw_data['test_data_neg'] = np.load("unbiased-test_arr_neg.npy")
raw_data['max_user'] = 15401
raw_data['max_item'] = 1001
batch_size = 8000
test_batch_size = 1000
display_itr = 1000

train_dataset = ImplicitDataset(raw_data['train_data'], raw_data['max_user'], raw_data['max_item'], name='Train')
test_dataset_pos = ImplicitDataset(raw_data['test_data_pos'], raw_data['max_user'], raw_data['max_item'])
test_dataset_neg = ImplicitDataset(raw_data['test_data_neg'], raw_data['max_user'], raw_data['max_item'])

# Define & Load Model

In [29]:
#Code to avoid tf using cached embeddings
import tensorflow as tf
tf.compat.v1.reset_default_graph()

In [30]:
cml_model = CML(batch_size=batch_size, max_user=train_dataset.max_user(), max_item=train_dataset.max_item(),
    dim_embed=50, l2_reg=0.001, opt='Adam', sess_config=None)
sampler = PairwiseSampler(batch_size=batch_size, dataset=train_dataset, num_process=4)
model_trainer = ImplicitModelTrainer(batch_size=batch_size, test_batch_size=test_batch_size,
                                     train_dataset=train_dataset, model=cml_model, sampler=sampler,
                                     eval_save_prefix="./yahoo",
                                     item_serving_size=500)
auc_evaluator = AUC()

In [31]:
cml_model.load("cml-yahoo")

INFO:tensorflow:Restoring parameters from bpr-yahoo


# Generate Raw Results

In [32]:
model_trainer._eval_manager = ImplicitEvalManager(evaluators=[auc_evaluator])
model_trainer._num_negatives = 200
model_trainer._exclude_positives([train_dataset, test_dataset_pos, test_dataset_neg])
model_trainer._sample_negatives(seed=10)

model_trainer._eval_save_prefix = "cml-yahoo-test-pos-unbiased"
model_trainer._evaluate_partial(test_dataset_pos)

model_trainer._eval_save_prefix = "cml-yahoo-test-neg-unbiased"
model_trainer._evaluate_partial(test_dataset_neg)

[Subsampling negative items]


100%|██████████| 15400/15400 [06:36<00:00, 38.88it/s]


{'AUC': [0.46238333333333326,
  0.5169397993311037,
  0.53285,
  0.4791666666666667,
  0.50285,
  0.49735,
  0.5094333333333333,
  0.5285000000000001,
  0.49663333333333337,
  0.5176677852348993,
  0.4606833333333333,
  0.44132107023411377,
  0.46606666666666674,
  0.4598,
  0.4731,
  0.48332775919732435,
  0.4625999999999999,
  0.4511705685618728,
  0.47403333333333325,
  0.4999333333333334,
  0.5061,
  0.5054,
  0.4870833333333333,
  0.47063333333333335,
  0.48975,
  0.5143311036789298,
  0.5243791946308726,
  0.6251174496644295,
  0.49700000000000005,
  0.5149333333333334,
  0.5057166666666667,
  0.5395500000000001,
  0.4956666666666667,
  0.5364833333333333,
  0.48818333333333336,
  0.5015436241610739,
  0.5018666666666667,
  0.4826677852348993,
  0.5264166666666666,
  0.48066666666666663,
  0.5008333333333334,
  0.50195,
  0.516989966555184,
  0.4722166666666666,
  0.48754999999999993,
  0.5148327759197324,
  0.4951,
  0.4871,
  0.46471666666666667,
  0.5227,
  0.4776666666666667,

# Evaluation

In [22]:
def eq(infilename, infilename_neg, trainfilename, gamma=-1.0, K=1):
    infile = open(infilename, 'rb')
    infile_neg = open(infilename_neg, 'rb')
    P = pickle.load(infile)
    infile.close()
    P_neg = pickle.load(infile_neg)
    infile_neg.close()
    NUM_NEGATIVES = P["num_negatives"]
    #
    for theuser in P["users"]:
        neg_items = list(P_neg["user_items"][theuser][NUM_NEGATIVES:])
        neg_scores = list(P_neg["results"][theuser][NUM_NEGATIVES:])
        P["user_items"][theuser] = list(neg_items) + list(P["user_items"][theuser][NUM_NEGATIVES:])
        P["results"][theuser] = list(neg_scores) + list(P["results"][theuser][NUM_NEGATIVES:])
    #
    Zui = dict()
    Ni = dict()
    # fill in dictionary Ni
    trainset = np.load(trainfilename)
    for i in trainset['item_id']:
        if i in Ni:
            Ni[i] += 1
        else:
            Ni[i] = 1
    del trainset
    # count #users with non-zero item frequencies
    nonzero_user_count = 0
    for theuser in P["users"]:
        pos_items = P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]
        for pos_item in pos_items:
            if pos_item in Ni:
                nonzero_user_count += 1
                break
    # fill in dictionary Zui
    for theuser in P["users"]:
        all_scores = np.array(P["results"][theuser])
        pos_items = P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]
        pos_scores = P["results"][theuser][len(P_neg["results"][theuser][NUM_NEGATIVES:]):]
        for i, pos_item in enumerate(pos_items):
            pos_score = pos_scores[i]
            Zui[(theuser, pos_item)] = float(np.sum(all_scores > pos_score))
    # calculate per-user scores
    sum_user_auc = 0.0
    sum_user_recall = 0.0
    for theuser in P["users"]:
        numerator_auc = 0.0
        numerator_recall = 0.0
        denominator = 0.0
        for theitem in P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]:
            if theitem not in Ni:
                continue
            pui = np.power(Ni[theitem], (gamma + 1) / 2.0)
            numerator_auc += (1 - Zui[(theuser, theitem)] / len(P["user_items"][theuser])) / pui
            # Calcolo il Recall a 1, vedi nota 6 paper
            if Zui[(theuser, theitem)] < K:
                numerator_recall += 1.0 / pui
            denominator += 1 / pui
        if denominator > 0:
            sum_user_auc += numerator_auc / denominator
            sum_user_recall += numerator_recall / denominator

    return {
        "auc"       : sum_user_auc / nonzero_user_count,
        "recall"    : sum_user_recall / nonzero_user_count
    }

In [33]:
eq("bpr-yahoo-test-pos-unbiased_evaluate_partial.pickle", "bpr-yahoo-test-neg-unbiased_evaluate_partial.pickle", "training_arr.npy", gamma=1.5)

{'auc': 0.6477791099000514, 'recall': 0.005725722479383864}

In [28]:
eq("cml-yahoo-test-pos-unbiased_evaluate_partial.pickle", "cml-yahoo-test-neg-unbiased_evaluate_partial.pickle", "training_arr.npy", gamma=1.5)

{'auc': 0.6582512516831484, 'recall': 0.01215594999877075}